In [ ]:
import os
import random
root = "/content/drive/MyDrive/dataset"
gen = os.walk(root)
list_of_folders = next(gen)[1]
list_of_folders.sort()

train = []
valid = []

for folder in list_of_folders:
  r1 = os.path.join(root, folder)
  g = os.walk(r1)
  
  for subfolder in next(g)[1]:
    r2 = (os.path.join(r1, subfolder))

    files = [f for f in os.listdir(r2) if os.path.isfile(os.path.join(r2, f))]

    for file_name in files:
      if random.uniform(0, 1) < 0.8:
        train.append([os.path.join(r2, file_name), 1 if subfolder == "fall" else 0])
      else:
        valid.append([os.path.join(r2, file_name), 1 if subfolder == "fall" else 0])

# 80 - 20 split to traning & validation data

In [ ]:
import pandas as pd

train_df = pd.DataFrame(train, columns=['path', 'fall'])
valid_df = pd.DataFrame(valid, columns=['path', 'fall'])
print(train_df)

                                                 path  fall
0   /content/drive/MyDrive/dataset/Anju/notFall/da...     0
1   /content/drive/MyDrive/dataset/Anju/notFall/da...     0
2   /content/drive/MyDrive/dataset/Anju/notFall/da...     0
3   /content/drive/MyDrive/dataset/Anju/notFall/da...     0
4   /content/drive/MyDrive/dataset/Anju/notFall/da...     0
..                                                ...   ...
80  /content/drive/MyDrive/dataset/dhirendraBhai/n...     0
81  /content/drive/MyDrive/dataset/dhirendraBhai/n...     0
82  /content/drive/MyDrive/dataset/dhirendraBhai/n...     0
83  /content/drive/MyDrive/dataset/dhirendraBhai/n...     0
84  /content/drive/MyDrive/dataset/dhirendraBhai/n...     0

[85 rows x 2 columns]


In [ ]:
train_df.fall.value_counts()

1    43
0    42
Name: fall, dtype: int64

In [ ]:
valid_df.fall.value_counts()

0    17
1    16
Name: fall, dtype: int64

In [ ]:
shapes = []
import torch

for i in range(len(train_df)):
  csv_path = train_df.iloc[i, 0]
  mat = pd.read_csv(csv_path)
  mat = [list(mat[x]) for x in ['ax', 'ay', 'az', 'gx', 'gy', 'gz']]
  mat = torch.tensor(mat)
  if mat.shape[1] < 10:
    print(csv_path)
  shapes.append(mat.shape[1])

print(shapes)
mat_min = min(shapes) # csv to tensor

[121, 124, 121, 120, 119, 122, 131, 108, 131, 128, 129, 134, 127, 132, 138, 134, 132, 138, 144, 136, 133, 137, 134, 136, 136, 135, 135, 128, 130, 129, 141, 138, 132, 143, 92, 137, 141, 142, 138, 137, 123, 94, 120, 128, 119, 130, 126, 128, 128, 133, 88, 130, 132, 134, 142, 134, 134, 121, 129, 133, 132, 134, 135, 132, 135, 133, 134, 133, 136, 127, 127, 129, 128, 130, 129, 131, 132, 136, 137, 134, 138, 139, 136, 137, 136]


In [ ]:
import os
import torch
import pandas as pd
from torch.utils.data import Dataset, IterableDataset

def mat_to_tensor(mat_path):
  mat = pd.read_csv(mat_path)
  mat = mat.dropna()
  mat = [list(mat[x]) for x in ['ax', 'ay', 'az', 'gx', 'gy', 'gz']]
  mat = torch.tensor(mat)
  return mat

class IMUSet(Dataset):
    def __init__(self, data_df):
        self.data_df = data_df

    def __len__(self):
        return len(self.data_df)

    def __getitem__(self, idx):
        mat_path = self.data_df.iloc[idx, 0]
        t = mat_to_tensor(mat_path)
        label = self.data_df.iloc[idx, 1]
        return t, label

In [ ]:
from torch.utils.data import DataLoader

imuset = IMUSet(train_df) # make dataset

train_dataloader = DataLoader(imuset, shuffle=True) 

In [ ]:
valid_dataset = IMUSet(valid_df)
valid_dataloader = DataLoader(valid_dataset, shuffle=True)

valid_features, valid_labels = next(iter(valid_dataloader))

valid_features.shape, valid_labels.shape

(torch.Size([1, 6, 121]), torch.Size([1]))

**LSTM**

In [ ]:
from torch import nn
import torch.nn.functional as F

class FallDetector(nn.Module):
    def __init__(self):
        super(FallDetector, self).__init__()
        self.lstm = nn.LSTM(input_size=6, hidden_size=5, num_layers=1)
        self.hidden2tag = nn.Linear(5, 1)

    def forward(self, seq):
        output, (h_n, c_n) = self.lstm(seq.view(len(seq), 1, -1))
        tag_space = self.hidden2tag(c_n.view(1, -1))
        # print(tag_space.data)
        tag_scores = torch.sigmoid(tag_space)
        return tag_scores.view(-1)

In [ ]:
import torch.optim as optim

model = FallDetector()
loss_function = nn.BCELoss() 
optimizer = optim.Adam(model.parameters())

In [ ]:
import numpy as np

from sklearn.metrics import confusion_matrix, accuracy_score, recall_score

def get_statistics(y_true, y_pred):
    y_pred = np.concatenate(tuple(y_pred)) > 0.5
    y_true = np.concatenate(tuple([[t for t in y] for y in y_true])).reshape(y_pred.shape)

    print("Accuracy: ", accuracy_score(y_true, y_pred))
    print("Recall: ", recall_score(y_true, y_pred))
    print(len(y_true))

    print("Confusion Matrix")
    print(confusion_matrix(y_true, y_pred))

In [ ]:
for epoch in range(50): 
    labels = []
    preds = []
    loss = []

    i = 0
    for seq, label in train_dataloader:
        model.zero_grad()

        seq = seq.reshape((-1, 6))
        tag_scores = model(seq.to(torch.float))
        preds.append(tag_scores.data)
        labels.append(label.to(torch.float).data)

        loss = loss_function(tag_scores, label.to(torch.float))
        loss.backward()
        optimizer.step()

        i += 1

    print("Epoch Done")
    get_statistics(labels, preds)

    # tag_scores[tag_scores > 0.4] = 1


Epoch Done
Accuracy:  0.6823529411764706
Recall:  0.7441860465116279
85
Confusion Matrix
[[26 16]
 [11 32]]
Epoch Done
Accuracy:  0.6705882352941176
Recall:  0.6976744186046512
85
Confusion Matrix
[[27 15]
 [13 30]]
Epoch Done
Accuracy:  0.7294117647058823
Recall:  0.813953488372093
85
Confusion Matrix
[[27 15]
 [ 8 35]]
Epoch Done
Accuracy:  0.6823529411764706
Recall:  0.7209302325581395
85
Confusion Matrix
[[27 15]
 [12 31]]
Epoch Done
Accuracy:  0.6588235294117647
Recall:  0.7441860465116279
85
Confusion Matrix
[[24 18]
 [11 32]]
Epoch Done
Accuracy:  0.7294117647058823
Recall:  0.8837209302325582
85
Confusion Matrix
[[24 18]
 [ 5 38]]
Epoch Done
Accuracy:  0.7058823529411765
Recall:  0.6511627906976745
85
Confusion Matrix
[[32 10]
 [15 28]]
Epoch Done
Accuracy:  0.6823529411764706
Recall:  0.7209302325581395
85
Confusion Matrix
[[27 15]
 [12 31]]
Epoch Done
Accuracy:  0.7411764705882353
Recall:  0.7674418604651163
85
Confusion Matrix
[[30 12]
 [10 33]]
Epoch Done
Accuracy:  0.77647

In [ ]:
labels = []
preds = []
loss = []
model.eval()

for seq, label in valid_dataloader:

    seq = seq.reshape((-1, 6))
    tag_scores = model(seq.to(torch.float))
    preds.append(tag_scores.data)
    labels.append(label.to(torch.float).data)

print("Epoch Done")
get_statistics(labels, preds)

Epoch Done
Accuracy:  0.6363636363636364
Recall:  0.375
33
Confusion Matrix
[[15  2]
 [10  6]]


In [ ]:
# torch.save(model.state_dict(), 'model_weights.pt')

In [ ]:
# from torch import nn
# import torch.nn.functional as F

# class FallDetector(nn.Module):
#     def __init__(self):
#         super(FallDetector, self).__init__()
#         self.lstm = nn.LSTM(6, 5, num_layers=1)
#         self.hidden2tag = nn.Linear(5, 1)

#     def forward(self, seq):
#         output, (h_n, c_n) = self.lstm(seq.view(len(seq), 1, -1))
#         tag_space = self.hidden2tag(c_n.view(1, -1))
#         # print(tag_space.data)
#         tag_scores = torch.sigmoid(tag_space)
#         return tag_scores.view(-1)

# model = FallDetector()
# model.load_state_dict(torch.load('model_weights.pt'))
# model.eval()